In [96]:
import pandas as pd                         #for data wrangling
import requests                             #for making http requests to target server
import yfinance as yf                       #for pulling financial information - income statement
import sys
from bs4 import BeautifulSoup as bs         #for parsing http response 
from datetime import datetime
from sqlalchemy import create_engine,text   #for writing pandas dataframe to mysql database

In [97]:
url = 'https://www.slickcharts.com/sp500'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36", 
    "Accept-Language": "en-US,en;q=0.9"
           }

In [98]:
page = requests.get(url, headers=headers)
print(page.status_code)

200


In [99]:
soup = bs(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <link href="/css/cosmo/bootstrap.min.css" rel="stylesheet"/>
  <style>
   .maxWidth {
    max-width: 1200px;
}
body {
    background-color: #FBFCEA;
    background-color: rgb(250, 250, 250);
}
  </style>
  <link href="https://fonts.googleapis.com" rel="preconnect"/>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <link href="https://fonts.googleapis.com/css2?family=Open+Sans:ital,wght@0,300;0,400;0,500;0,600;0,700;0,800;1,300;1,400;1,500;1,600;1,700;1,800&amp;display=swap" rel="stylesheet"/>
  <title>
   S&amp;P 500 Companies by Weight
  </title>
  <link crossorigin="" href="https://a.pub.network/" rel="preconnect">
   <link crossorigin="" href="https://b.pub.network/" rel="preconnect">
    <link crossorigin="" href="https://c.pub.network/" rel="preconnect">
     <link crossorigin="" href="https://d.p

In [100]:
table = soup.find('table', class_='table')
print(table.prettify())

<table class="table table-hover table-borderless table-sm">
 <thead>
  <tr>
   <th scope="col">
    #
   </th>
   <th scope="col" style="width: 30%">
    Company
   </th>
   <th scope="col">
    Symbol
   </th>
   <th scope="col">
    Weight
   </th>
   <th scope="col">
    Price
   </th>
   <th scope="col">
    Chg
   </th>
   <th scope="col">
    % Chg
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    1
   </td>
   <td style="max-width: 120px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;">
    <a href="/symbol/NVDA">
     Nvidia Corp
    </a>
   </td>
   <td>
    <a href="/symbol/NVDA">
     NVDA
    </a>
   </td>
   <td>
    7.16%
   </td>
   <td class="text-nowrap">
    <img alt="" src="/img/up.gif"/>
    142.00
   </td>
   <td class="text-nowrap" style="color: green">
    0.05
   </td>
   <td class="text-nowrap" style="color: green">
    (0.04%)
   </td>
  </tr>
  <tr>
   <td>
    2
   </td>
   <td style="max-width: 120px; overflow: hidden; text-overflow: e

In [101]:
symbols = table.find_all('a')
print(symbols)

[<a href="/symbol/NVDA">Nvidia Corp</a>, <a href="/symbol/NVDA">NVDA</a>, <a href="/symbol/AAPL">Apple Inc.</a>, <a href="/symbol/AAPL">AAPL</a>, <a href="/symbol/MSFT">Microsoft Corp</a>, <a href="/symbol/MSFT">MSFT</a>, <a href="/symbol/AMZN">Amazon.com Inc</a>, <a href="/symbol/AMZN">AMZN</a>, <a href="/symbol/META">Meta Platforms, Inc. Class A</a>, <a href="/symbol/META">META</a>, <a href="/symbol/GOOGL">Alphabet Inc. Class A</a>, <a href="/symbol/GOOGL">GOOGL</a>, <a href="/symbol/TSLA">Tesla, Inc.</a>, <a href="/symbol/TSLA">TSLA</a>, <a href="/symbol/BRK.B">Berkshire Hathaway Class B</a>, <a href="/symbol/BRK.B">BRK.B</a>, <a href="/symbol/GOOG">Alphabet Inc. Class C</a>, <a href="/symbol/GOOG">GOOG</a>, <a href="/symbol/AVGO">Broadcom Inc.</a>, <a href="/symbol/AVGO">AVGO</a>, <a href="/symbol/JPM">Jpmorgan Chase &amp; Co.</a>, <a href="/symbol/JPM">JPM</a>, <a href="/symbol/LLY">Eli Lilly &amp; Co.</a>, <a href="/symbol/LLY">LLY</a>, <a href="/symbol/UNH">Unitedhealth Group In

In [102]:
tickers = list(set([symbol.get('href').split('/')[-1] for symbol in symbols]))
len(tickers)

502

In [105]:
def get_stock_prices(index, ticker_list):
    data_parsed = {}
    for ticker in ticker_list:
        try:
            yf_ticker = yf.Tickers(ticker)
            data = yf_ticker.history(period="1d", group_by= 'tickers')
            data = data.reset_index()
            data_dict = data.to_dict(orient='records')
            data_parsed[ticker] = {
                'Date' : data_dict[0][('Date', '')].strftime('%Y-%m-%d'),
                'Ticker' : ticker,
                'Stock Index': index.upper(),
                'Open' : round(data_dict[0][(ticker, 'Open')], 5),
                'High' : round(data_dict[0][(ticker, 'High')], 5),
                'Low' : round(data_dict[0][(ticker, 'Open')], 5),
                'Close' : round(data_dict[0][(ticker, 'Open')], 5),
                'Volume' : data_dict[0][(ticker, 'Volume')],
                'Dividends' : data_dict[0][(ticker, 'Dividends')],
                'Stock Splits' : data_dict[0][(ticker, 'Stock Splits')]        
            }
        except Exception as e:
            print(e)
    return data_parsed

In [106]:
d = get_stock_prices('sp500', tickers)
d

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

list index out of range


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

list index out of range


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

list index out of range


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'JCI': {'Date': '2024-11-22',
  'Ticker': 'JCI',
  'Stock Index': 'SP500',
  'Open': 83.77,
  'High': 84.17,
  'Low': 83.77,
  'Close': 83.77,
  'Volume': 4360700,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'QCOM': {'Date': '2024-11-22',
  'Ticker': 'QCOM',
  'Stock Index': 'SP500',
  'Open': 154.78999,
  'High': 156.88,
  'Low': 154.78999,
  'Close': 154.78999,
  'Volume': 5640800,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'CCI': {'Date': '2024-11-22',
  'Ticker': 'CCI',
  'Stock Index': 'SP500',
  'Open': 105.31,
  'High': 105.8,
  'Low': 105.31,
  'Close': 105.31,
  'Volume': 1541900,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'TER': {'Date': '2024-11-22',
  'Ticker': 'TER',
  'Stock Index': 'SP500',
  'Open': 105.12,
  'High': 108.25,
  'Low': 105.12,
  'Close': 105.12,
  'Volume': 1651600,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'SW': {'Date': '2024-11-22',
  'Ticker': 'SW',
  'Stock Index': 'SP500',
  'Open': 56.5,
  'High': 56.986,
  'Low': 56.5,
  'Close': 56.5,
  

In [108]:
sys.getsizeof(d)

13056

In [109]:
sys.getsizeof(pd.DataFrame(d.values()))

122498

In [30]:
e = d.to_dict(orient='records')
e

[{('AAPL', 'Open'): 228.05999755859375,
  ('AAPL', 'High'): 230.72000122070312,
  ('AAPL', 'Low'): 228.05999755859375,
  ('AAPL', 'Close'): 229.8699951171875,
  ('AAPL', 'Volume'): 38153300,
  ('AAPL', 'Dividends'): 0.0,
  ('AAPL', 'Stock Splits'): 0.0}]

In [124]:
import csv
import os
def load_data(data):
        csv_file = 'stock_prices'
        headers = [
            'Date',
            'Ticker',
            'Stock Index',
            'Open',
            'High',
            'Low',
            'Close',
            'Volume',
            'Dividends',
            'Stock Splits'            
        ]
        file_exists = os.path.isfile(csv_file)
        with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            if not file_exists:
                writer.writeheader()
            writer.writerows(data.values()) 

In [125]:
load_data(d)